In [56]:
filtered_df.columns

Index(['Job Code', 'DESCRIPTION', 'CODE'], dtype='object')

In [60]:
filtered_df = filtered_df.drop('Job Code',axis=True)

In [62]:
filtered_df.to_excel('TrainingFile.xlsx')

In [1]:
import pandas as pd

def all_in_one(location, x):
    # Load the Excel file
    df = pd.read_excel(location)

    # Function to determine the indentation level of a string
    def indent_level(a):
        return len(a) - len(a.lstrip())

    # Function to assign hierarchy levels based on indentation
    def assign_hierarchy(df, x):
        numbers = []
        for i in df[x]:
            numbers.append(indent_level(i) // 2)
        df['Hierarchy'] = numbers
        return df

    # Process each row in the dataframe to build the hierarchy
    def concatenator(df, x):
        # Initialize variables for processing the hierarchy
        cnt = -1
        hierarchy = []
        largest = []
        concat = []
        for i in df['Hierarchy']:
            cnt += 1
            # A heading is encountered
            if i > 0:
                concat.append('-1')
                # Check if this level of heading hierarchy has been encountered before; if not, append the new level
                if not largest or i > max(largest):
                    largest.append(i)
                    hierarchy.append(' ' + df[x][cnt])
                # If it has been encountered before, delete any following hierarchy and replace the same level hierarchy with newly encountered heading
                if i in largest:
                    index = largest.index(i)
                    hierarchy[index] = ' ' + df[x][cnt]
                    del hierarchy[index+1:]
                    del largest[index+1:]
            # For a normal record
            elif i == 0:
                concat.append(' '.join(hierarchy) + ' ' + df[x][cnt])
        df['Concat'] = concat    

    # Function to order the DataFrame based on the hierarchy
    def order(df):
        ord_lst = []
        cnt = 0 
        for j in range(len(df['Hierarchy'])):
            item = df['Hierarchy'][j]
            if item == 0:
                cnt += 1
                ord_lst.append(cnt)
            else:
                cnt = 0
                ord_lst.append(0)
        df['Order'] = ord_lst

    # Assign hierarchy levels to the dataframe
    assign_hierarchy(df, x)

    # Concatenate based on the hierarchy
    concatenator(df, x)

    # Order the dataframe
    order(df)
    
    # Return the final DataFrame
    return df

In [68]:
# Example usage
df_result = all_in_one('Book2.xlsx', 'DESCRIPTION')
print(df_result)

      Job Code                                        DESCRIPTION CODE  \
0     SA24M005                             Contractual Milestones  NaN   
1     SA24M005                                         Portion- I  NaN   
2     SA24M005  Complete Mobilization and Commence the work- P...  NaN   
3     SA24M005  Technical Completion of the Project ( TCC )- P...  NaN   
4     SA24M005  Preliminary Acceptance of the Project ( PAC )-...  NaN   
...        ...                                                ...  ...   
7160  SA24M005                 FAC (Final Acceptance Certificate)  NaN   
7161  SA24M005           Remote Normalization Before Energization  NaN   
7162  SA24M005              Normalization Before Energization EHV  NaN   
7163  SA24M005                               Energization for EHV  NaN   
7164  SA24M005          Post Energization In Service Test for EHV  NaN   

      Hierarchy                                             Concat  Order  
0             1                    

In [88]:
df_result

,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order
0,SA24M005,Contractual Milestones,NaN,1,-1,0
1,SA24M005,Portion- I,NaN,2,-1,0
2,SA24M005,Complete Mobilization and Commence the work- P...,NaN,0,Contractual Milestones Portion- I Comp...,1
3,SA24M005,Technical Completion of the Project ( TCC )- P...,NaN,0,Contractual Milestones Portion- I Tech...,2
4,SA24M005,Preliminary Acceptance of the Project ( PAC )-...,NaN,0,Contractual Milestones Portion- I Prel...,3
...,...,...,...,...,...,...
7160,SA24M005,FAC (Final Acceptance Certificate),NaN,0,Project Portion- II (Construction of 380kV ...,3
7161,SA24M005,Remote Normalization Before Energization,NaN,1,-1,0
7162,SA24M005,Normalization Before Energization EHV,NaN,0,Remote Normalization Before Energization No...,1
7163,SA24M005,Energization for EHV,NaN,0,Remote Normalization Before Energization En...,2


In [95]:
df_result.to_excel("check2.xlsx")

In [97]:
# Filter rows where the 'Code' column is not null
filtered_df = df_result[df_result['CODE'].notna()]

In [98]:
filtered_df.to_excel('datasets-usefull/training-dataset-finalized.xlsx')

In [99]:
df3 = pd.read_excel('datasets-usefull/training-dataset-finalized.xlsx')

In [100]:
lst = []
for j in df3['CODE']:
    lst.append(j[0])
df3['Type'] = lst

In [101]:
df3[4:10]

,Unnamed: 0,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order,Type
4,70,SA24M005,FINAL Approval,E000E0DCS00E5,0,Project Portion- I (Construction of North Q...,5,E
5,72,SA24M005,Drafting / Preparation,E000E0MMM00E1,0,Project Portion- I (Construction of North Q...,1,E
6,73,SA24M005,Submission,E000E0MMM00E2,0,Project Portion- I (Construction of North Q...,2,E
7,74,SA24M005,Review / Evaluation,E000E0MMM00E3,0,Project Portion- I (Construction of North Q...,3,E
8,75,SA24M005,Approval with Comments,E000E0MMM00E4,0,Project Portion- I (Construction of North Q...,4,E
9,76,SA24M005,FINAL Approval,E000E0MMM00E5,0,Project Portion- I (Construction of North Q...,5,E


In [102]:
lst2 = []
cnt = -1
for j in df3['Order']:
    cnt+=1
    if j >=1 and j <10:
        lst2.append("00"+df3['Type'][cnt]+str(j))
    if j >=10 and j<100:
        lst2.append("0"+df3['Type'][cnt]+str(j))
    if j>=100 and j <1000:
        lst2.append(df3['Type'][cnt]+str(j))
    if j >=1000:
        lst2.append(df3['Type'][cnt]+str(j))

In [103]:
df3['LastFour'] = lst2

In [87]:
df3.to_excel('datasets/training-data-finalized-all-required.xlsx')

In [105]:

# Function to extract the middle part of the code
def extract_middle_part(code):
    return code[4:-4]

# Apply the function to the relevant column and create a new column
df3['MiddlePart'] = df3['CODE'].apply(extract_middle_part)


In [109]:
df3.to_excel('datasets-usefull/training-data.xlsx')

In [112]:
dict ={'P':'Procurement', 'C':'Construction','E':'Engineering', 'I':'Inspection and Delivery' }
name_lst = []
for i in df3['Type']:
    name_lst.append(dict[i])
df3['Category'] = name_lst
    

In [114]:
df3.to_excel('datasets-usefull/training-data-categories.xlsx')